# HW2: Part 1 - Primal-dual Interior-point method

In [4]:
import numpy as np, pandas as pd, scipy.optimize as sopt
import matplotlib.pyplot as plt
import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 1. Method implementation

In [2]:
# interior-point method
class InteriorPoint:
    def __init__(self):
        pass
    
    def solve(self, A, b, c):
        pass
    
    def _solve(self, A, b, c):
        pass

# remove unnecesary equations
# construct LP (P')
# construct dual (D')
# find initial iterate
# procees with iterative improvements
# when mu small enought
# calculate stopping parameter

# logging with observer

## 2. Diet problem

A man does not live by bread alone - apparently vegetable oil is also needed.

In [28]:
# print-out optimal diet
def display_diet(f_opt, x_opt):
    print(f"min. cost: {round(f_opt, 2)}")
    print(f"CH: {round(np.array([ 18,   48,   5,    1,   5,    0,    0,   8]).dot(x_opt), 2)}")
    print(f"PR: {round(np.array([  2,   11,   3,   13,   3,    0,   15,   1]).dot(x_opt), 2)}")
    print(f"FT: {round(np.array([  0,    5,   3,   10,   3,  100,   30,   1]).dot(x_opt), 2)}")
    print(f"EN: {round(np.array([ 77,  270,  60,  140,  61,  880,  330,  32]).dot(x_opt), 2)}")
    print(20 * "--")
    print(20 * "--")
    foods = ["potatoes", "bread", "milk", "eggs", "yogurt", "veg. oil", "beef", "strawbrs"]
    for food, x in zip(foods, x_opt):
        print(f"{food}: {round(100 * x)}g")

# setup diet LP problem
cost = [10, 22, 15, 45, 40, 20, 87, 21]
constr_A = [[ 18,   48,   5,    1,   5,    0,    0,   8],
            [-18,  -48,  -5,   -1,  -5,    0,    0,  -8],
            [  2,   11,   3,   13,   3,    0,   15,   1],
            [ -2,  -11,  -3,  -13,  -3,    0,  -15,  -1],
            [  0,    5,   3,   10,   3,  100,   30,   1],
            [  0,   -5,  -3,  -10,  -3, -100,  -30,  -1],
            [ 77,  270,  60,  140,  61,  880,  330,  32],
            [-77, -270, -60, -140, -61, -880, -330, -32]]
constr_b = [370, -250, 170, -50, 90, -50, 2400, -2200]

# use scipy solver
bnd = len(cost) * [(0, float("inf"))]
res = sopt.linprog(c=cost, A_ub=constr_A, b_ub=constr_b, bounds=bnd, method="interior-point")

display_diet(res.fun, res.x)

# TODO: use my interior-point method

min. cost: 148.83
CH: 299.04
PR: 68.53
FT: 90.0
EN: 2200.0
----------------------------------------
----------------------------------------
potatoes: 0g
bread: 623g
milk: 0g
eggs: 0g
yogurt: 0g
veg. oil: 59g
beef: 0g
strawbrs: 0g


## 3. Analytic center

In [ ]:
# analytic center - theory